In [1]:
import pandas as pd
import numpy as np
from IPython.display import display, HTML
import os

notebook_path = os.path.abspath("adhoc.ipynb")
notebook_dir = str(os.path.dirname(notebook_path)).replace("\\\\", "/").replace("\\", "/")
output_folder = f"{notebook_dir}/../output"
raw_output = f"{output_folder}/raw"
curated_output = f"{output_folder}/curated"
semantic_output = f"{output_folder}/semantic"

nyse_tickers = f"{notebook_dir}/../wsb/nyse-listed.csv"
nasdaq_tickers = f"{notebook_dir}/../wsb/nasdaq-listed.csv"
curated_stockticker = f"{curated_output}/StockTicker.csv"
curated_dailydiscussion = f"{curated_output}/DailyDiscussion.csv"

# output_html = f"{notebook_dir}/../index.html"

In [2]:
# https://stackoverflow.com/questions/34478398/import-local-function-from-a-module-housed-in-another-directory-with-relative-im
# import os
import sys
module_path = os.path.abspath(os.path.join('../wsb'))
if module_path not in sys.path:
    sys.path.append(module_path)

import models

models.HTML()

[Errno 2] No such file or directory: 'template.html'
